In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir("".join((os.getcwd().split("PCNN")[0], "/PCNN/src/core")))
import run_core as rc
import utils_core as utc

%load_ext autoreload
%autoreload 2

#plt.style.use('classic')

logger = utc.setup_logger("M", level=1)
logger()

2024-11-19 14:10:41 | 


## simulation

In [2]:
sim_settings = rc.sim_settings
agent_settings = rc.agent_settings
model_params = rc.model_params

In [3]:
load = False

In [4]:
rc.edit_logger(level=3)

In [7]:
if load:
    sim_settings, agent_settings, model_params = utc.load_model_settings(idx=None,
                                                                         verbose=True)

    if sim_settings is None:
        load = False
        

count = 0
while True:

    logger(f"[round {count}]", level=0)

    if not load:
        sim_settings["seed"] = np.random.randint(0, 1000)
        sim_settings["init_position"] = np.random.uniform(0.1, 0.9, 2)
        sim_settings["rw_fetching"] = "deterministic"
        sim_settings["rw_position"] = np.random.uniform(0.1, 0.9, 2)
        sim_settings["rw_radius"] = 0.05
        sim_settings["plot_interval"] = 5
        sim_settings["speed"] = 0.04
        sim_settings["max_duration"] = 1000

        agent_settings["max_depth"] = 20
        agent_settings["exp_weights"] = np.array([-1., 0.2, -1., 0.2, 0.4]),

    model = rc.main(sim_settings=sim_settings,
         agent_settings=agent_settings,
         model_params=model_params)

    count += 1


2024-11-19 14:11:35 | [round 0]
2024-11-19 14:11:35 | room: square
2024-11-19 14:11:35 | plot_interval: 5
2024-11-19 14:11:35 | 1000
2024-11-19 14:11:35 | N=80
2024-11-19 14:11:35 | Nj=169
2024-11-19 14:11:35 | Reward(deterministic, static)
2024-11-19 14:12:45 | [round 1]
2024-11-19 14:12:45 | room: square
2024-11-19 14:12:45 | plot_interval: 5
2024-11-19 14:12:45 | 1000
2024-11-19 14:12:45 | N=80
2024-11-19 14:12:45 | Nj=169
2024-11-19 14:12:45 | Reward(deterministic, static)
2024-11-19 14:13:47 | [round 2]
2024-11-19 14:13:47 | room: square
2024-11-19 14:13:47 | plot_interval: 5
2024-11-19 14:13:47 | 1000
2024-11-19 14:13:47 | N=80
2024-11-19 14:13:47 | Nj=169
2024-11-19 14:13:47 | Reward(deterministic, static)


KeyboardInterrupt: 

In [10]:
model.__str__()

'<mod_core.Brain object at 0x13015b290>'